### K-Means para aprendizaje semi-supervisado.

El aprendizaje semi-supervisado (o *Semi-supervised Learning*) comprende el conjunto de técnicas que nos permiten entrenar modelos con datasets parcialmente etiquetados. En esta sección vamos a ver un ejemplo de como podemos aplicar esta técnica con el dataset MNIST y usando *K-Means*. Empezamos descargando el dataset.

In [ ]:
from sklearn.datasets import fetch_lfw_people


# Cargar el dataset con imágenes de al menos 70 rostros por persona (puedes ajustar min_faces_per_person)
lfw_dataset = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# Acceder a los datos
X = lfw_dataset.data          # Imágenes en formato vectorizado (flattened)
y = lfw_dataset.target        # Etiquetas (persona)
target_names = lfw_dataset.target_names  # Nombres de las personas

# Verificar las dimensiones
print("Shape de X:", X.shape)
print("Número de clases:", len(target_names))


In [ ]:
import matplotlib.pyplot as plt

# Mostrar las primeras 10 imágenes del dataset
fig, axes = plt.subplots(1, 10, figsize=(15, 8))
for i in range(10):
    axes[i].imshow(lfw_dataset.images[i], cmap="gray")
    axes[i].axis("off")
    axes[i].set_title(target_names[y[i]][:10])  # mostrar el nombre
plt.tight_layout()
plt.show()


In [ ]:
#preparar el dataset
from sklearn.model_selection import train_test_split
import numpy as np

# Cargamos el dataset
lfw = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

X = lfw.data  # características (imagenes aplanadas)
y = lfw.target  # etiquetas (personas)
target_names = lfw.target_names  # nombres de personas

print("Shape de X:", X.shape)  # ejemplo: (1288, 1850)
print("Cantidad de clases:", len(target_names))

# Dividimos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [ ]:
from sklearn.cluster import KMeans

k = 50  # número de clusters
kmeans = KMeans(n_clusters=k, random_state=42)
X_faces_dist = kmeans.fit_transform(X_train)  # distancia a cada centroide

print("Shape de distancias:", X_faces_dist.shape)


In [ ]:
idxs = np.argmin(X_faces_dist, axis=0)  # índice más cercano a cada centroide
X_representative_faces = X_train[idxs]  # imágenes representativas

print("Shape:", X_representative_faces.shape)


In [ ]:
import matplotlib.pyplot as plt

h, w = lfw.images.shape[1:3]  # alto y ancho original

plt.figure(figsize=(15, 6))
for index, face in enumerate(X_representative_faces):
    plt.subplot(5, 10, index + 1)
    plt.imshow(face.reshape(h, w), cmap="gray")
    plt.title(f"{index+1}", fontsize=8)
    plt.axis('off')
plt.suptitle("Imágenes más representativas por cluster", fontsize=16)
plt.show()


In [ ]:
# Obtener las etiquetas reales de las imágenes más representativas
y_representative_faces = y_train[idxs]


In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg2 = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
%time log_reg2.fit(X_representative_faces, y_representative_faces)

# Evaluamos el modelo en el conjunto de prueba
print("Precisión usando solo imágenes representativas:", log_reg2.score(X_test, y_test))




In [ ]:
log_reg_random = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
%time log_reg_random.fit(X_train[:50], y_train[:50])

# Evaluamos el modelo en el conjunto de prueba
print("Precisión usando 50 imágenes aleatorias:", log_reg_random.score(X_test, y_test))



In [ ]:
# Propagación de etiquetas representativas a todo el conjunto de entrenamiento
y_train_propagated = np.empty(len(X_train))
for i in range(k):
    y_train_propagated[kmeans.labels_ == i] = y_representative_faces[i]

# Entrenamiento con las etiquetas propagadas
log_reg3 = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
%time log_reg3.fit(X_train[:1000], y_train_propagated[:1000])

print("Precisión con 1000 muestras semi-etiquetadas:", log_reg3.score(X_test, y_test))


## Aprendizaje Activo

In [ ]:
# Calculamos las probabilidades de las primeras 1000 muestras
probas = log_reg3.predict_proba(X_train[:1000])

# Obtenemos el índice de la clase más probable para cada muestra
labels_ixs = np.argmax(probas, axis=1)

# Obtenemos la confianza (probabilidad máxima) de cada predicción
confidences = probas[np.arange(len(probas)), labels_ixs]



In [ ]:
# Ordenamos por confianza (de menor a mayor)
sorted_ixs = np.argsort(confidences)

k = 50  # cantidad de muestras "inciertas" a revisar

# Obtenemos las 50 imágenes más inciertas
X_lowest = X_train[:1000][sorted_ixs[:k]]
y_lowest = y_train[:1000][sorted_ixs[:k]]


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
for i in range(10):
    for j in range(5):
        idx = i * 5 + j
        image = X_lowest[idx].reshape(50, 37)
        plt.subplot(10, 5, idx + 1)
        plt.imshow(image, cmap="gray")
        plt.axis("off")
plt.suptitle("50 imágenes más inciertas según el modelo")
plt.tight_layout()
plt.show()


In [ ]:
log_reg_active = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)

# Entrenamos con las 50 muestras inciertas (suponemos que ya están etiquetadas correctamente)
%time log_reg_active.fit(X_lowest, y_lowest)

# Evaluamos en el conjunto de prueba
print("Precisión usando aprendizaje activo (50 imágenes inciertas):", log_reg_active.score(X_test, y_test))
